In [1]:
# Temel
import pandas as pd
import numpy as np

# Görselleştirme
import matplotlib.pyplot as plt
import seaborn as sns

# İstatistik
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan, normal_ad
from statsmodels.stats.stattools import jarque_bera

In [6]:
wdi = pd.read_csv("WDI_CSV_2025_07_02/WDICSV.csv")

# Geniş (wide) yapıyı uzun (long) formata dönüştürme
wdi_long = wdi.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    var_name="Year",
    value_name="Value"
)

# Yıl sütununu sayıya çevirme
wdi_long["Year"] = pd.to_numeric(wdi_long["Year"], errors="coerce")

In [17]:
from functools import reduce


# === Kullanılacak göstergeler ===
indicators = {
    # Yaşam süresi
    "SP.DYN.LE00.IN": "life_expectancy",

    # Ekonomik göstergeler
    "NY.GDP.PCAP.CD": "gdp_per_capita",
    "SI.POV.DDAY": "poverty_rate",
    "SL.UEM.TOTL.ZS": "unemployment_rate",

    # Beslenme ve sağlık
    "SH.STA.MMRT": "maternal_mortality",
    "SH.DYN.MORT": "under5_mortality",
    "SH.STA.BRTC.ZS": "birth_rate",
}


# === Her göstergeyi ayrı DataFrame olarak çıkar ve yeniden adlandır ===
dfs = []
for code, name in indicators.items():
    df_i = wdi_long[wdi_long["Indicator Code"] == code][
        ["Country Name", "Country Code", "Year", "Value"]
    ].rename(columns={"Value": name})
    dfs.append(df_i)

# === DataFrame'leri Ülke ve Yıl bazında birleştir ===
merged = reduce(
    lambda left, right: pd.merge(left, right, on=["Country Name", "Country Code", "Year"], how="inner"),
    dfs
)

# === Eksik değerleri temizle ===
merged = merged.dropna()

# === Log-transform (örnek) — kişi başı gelir için ===
merged["log_gdp_per_capita"] = np.log(merged["gdp_per_capita"])

print("✅ Veri hazır:", merged.shape)
merged

✅ Veri hazır: (1281, 11)


,Country Name,Country Code,Year,life_expectancy,gdp_per_capita,poverty_rate,unemployment_rate,maternal_mortality,under5_mortality,birth_rate,log_gdp_per_capita
8330,Canada,CAN,1991,77.593415,21846.793047,1.2,10.316,8.0,8.0,99.1,9.991809
8382,Hungary,HUN,1991,69.377073,3361.222680,2.2,8.500,24.0,16.5,99.4,8.120060
8442,Norway,NOR,1991,76.980732,28596.933004,0.5,5.407,8.0,7.9,99.3,10.261055
8501,United States,USA,1991,75.365854,24342.258905,0.5,6.800,12.0,10.9,99.2,10.099969
8568,Argentina,ARG,1992,72.096000,6789.996111,2.5,6.360,65.0,27.6,95.8,8.823206
...,...,...,...,...,...,...,...,...,...,...,...
16422,Norway,NOR,2021,83.163415,93072.892512,0.2,4.359,2.0,2.3,99.2,11.441138
16429,Peru,PER,2021,71.596000,6826.202556,5.2,5.097,141.0,16.4,94.9,8.828524
16431,Poland,POL,2021,75.351220,18635.508881,0.1,3.268,2.0,4.4,99.6,9.832824
16448,Slovak Republic,SVK,2021,74.614634,22132.448303,0.3,6.894,12.0,6.0,98.0,10.004800


In [19]:
import statsmodels.formula.api as smf

print("Veri boyutu:", merged.shape)
print("Sütunlar:", merged.columns.tolist())

formula = """
life_expectancy ~ log_gdp_per_capita
                 + poverty_rate
                 + unemployment_rate
                 + maternal_mortality
                 + under5_mortality
                 + birth_rate
"""

model_full = smf.ols(formula=formula, data=merged).fit()

print("\n--- Çok değişkenli model ---")
print(model_full.summary())

Veri boyutu: (1281, 11)
Sütunlar: ['Country Name', 'Country Code', 'Year', 'life_expectancy', 'gdp_per_capita', 'poverty_rate', 'unemployment_rate', 'maternal_mortality', 'under5_mortality', 'birth_rate', 'log_gdp_per_capita']

--- Çok değişkenli model ---
                            OLS Regression Results                            
Dep. Variable:        life_expectancy   R-squared:                       0.838
Model:                            OLS   Adj. R-squared:                  0.838
Method:                 Least Squares   F-statistic:                     1101.
Date:                Tue, 28 Oct 2025   Prob (F-statistic):               0.00
Time:                        17:10:06   Log-Likelihood:                -2956.2
No. Observations:                1281   AIC:                             5926.
Df Residuals:                    1274   BIC:                             5963.
Df Model:                           6                                         
Covariance Type:            nonr

# 📝 Yaşam Süresi ve Sosyo-Ekonomik Faktörler: Regresyon Analizi Raporu

## 1. Modelin Amacı
Bu model, ülkelerin yaşam süresini etkileyen faktörleri anlamayı amaçlamaktadır. Özellikle:

- Kişi başına düşen gelir  
- Yoksulluk oranı  
- İşsizlik oranı  
- Anne ve 5 yaş altı çocuk ölüm oranları  
- Doğum hızı  

gibi sosyal ve ekonomik değişkenlerin **yaşam süresine etkisi** incelenmiştir.

---

## 2. Modelin Genel Performansı

- **R² = 0.838**  
  - Model, yaşam süresindeki varyansın yaklaşık **%84’ünü açıkladığını** gösteriyor.  
  - Bu değişkenler birlikte yaşam süresinin çoğunu tahmin edebiliyor.

- **F-statistic = 1101, p < 0.001**  
  - Modelin genel olarak istatistiksel olarak anlamlı olduğunu gösterir.  
  - Modeldeki değişkenlerin topluca yaşam süresi üzerinde etkisi var.

- **Durbin-Watson ≈ 1.855**  
  - Artıklar (tahmin hataları) bağımsız, yani modelde otokorelasyon sorunu yok.

- **Cond. No = 2810**  
  - Bazı değişkenler arasında yüksek ilişki (multicollinearity) olabilir.  
  - Katsayılar hassas olabilir; küçük değişiklikler tahminleri etkileyebilir.

---

## 3. Katsayılar ve Yorumu

| Değişken | Katsayı | P-değeri | Anlamı |
|-----------|---------|----------|--------|
| **Intercept** | 58.14 | <0.001 | Tüm değişkenler 0 olduğunda beklenen yaşam süresi (teorik başlangıç değeri). |
| **log_gdp_per_capita** | 2.53 | <0.001 | Kişi başına düşen gelir arttıkça yaşam süresi artıyor. Her bir birim artış yaşam süresini yaklaşık 2.5 yıl artırıyor. |
| **poverty_rate** | 0.046 | <0.001 | Yoksulluk oranı arttığında yaşam süresi hafif artıyor gibi görünüyor. Veri veya diğer değişkenlerle ilişkiden kaynaklı olabilir. |
| **unemployment_rate** | 0.003 | 0.799 | Anlamlı değil; işsizlik oranının yaşam süresi üzerinde etkisi yok gibi. |
| **maternal_mortality** | -0.0098 | <0.001 | Anne ölüm oranı arttıkça yaşam süresi azalıyor (beklenen ilişki). |
| **under5_mortality** | -0.108 | <0.001 | 5 yaş altı ölüm oranı arttıkça yaşam süresi ciddi şekilde düşüyor. En güçlü negatif etki burada. |
| **birth_rate** | -0.047 | <0.001 | Doğum hızı arttıkça yaşam süresi azalıyor. Çok çocuklu toplumlarda sağlık ve yaşam süresi baskılanabiliyor. |

---

## 4. Modelin Ana Mesajları

1. **Ekonomik refah önemli:** Kişi başına gelir arttıkça yaşam süresi belirgin şekilde artıyor.  
2. **Sağlık göstergeleri kritik:** Anne ve çocuk ölüm oranları yaşam süresinin en güçlü negatif belirleyicileri.  
3. **Bazı beklenmedik sonuçlar:** Yoksulluk oranının pozitif etkisi, veri ve model kontrolünden kaynaklı olabilir ve dikkatle yorumlanmalı.  
4. **Sosyal politika için ipuçları:** Doğum hızı ve sağlık göstergelerindeki iyileşmeler, yaşam süresini artırmada etkili.

---

## 5. Modelin Gücü ve Sınırları

- **Güçlü yönleri:**  
  - Yüksek R² ile yaşam süresini çok iyi açıklıyor.  
  - Sosyal ve ekonomik faktörleri tek modelde birleştiriyor.

- **Sınırlamaları:**  
  - Cond. No yüksek → bazı katsayılar hassas, değişkenler birbirine bağlı olabilir.  
  - Bazı ters sonuçlar (poverty_rate) veri ve modelleme sınırlamalarından kaynaklanabilir.

---

## 6. Özet

> Bu model, yaşam süresini belirleyen temel ekonomik ve sağlık göstergelerini güçlü bir şekilde yakalıyor.  
> Politika yapıcılar için en önemli mesaj: **ekonomik refahın artırılması, anne ve çocuk sağlığının iyileştirilmesi ve aşırı doğum oranlarının kontrolü**, yaşam süresini artırmada kritik öneme sahip.
